<a href="https://colab.research.google.com/github/scumulder/Machine-Learning-Spring-2019-UCF/blob/master/Homework%202/CNN3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**About this CNN:**


---


*   Increased epochs (epochs = 50)
*   Data Augmentation 
*   Dropout
*   {Conv2D + ReLU} -> {Conv2D + ReLU} -> maxpool -> {Conv2D + ReLU} -> {Conv2D + ReLU} -> maxpool -> Fully Connected Layers
*   Change in filter sizes of Conv2D layers


---




**Loading the CIFAR10 data set:**


In [1]:
from keras.datasets import cifar10
(train_images_original, train_labels_original), (test_images, test_labels) = cifar10.load_data()

#X_train = train_images
#Y_train = train_labels

#X_test = test_images
#Y_test = test_labels

Using TensorFlow backend.


**Split Training Data into Training Set and Validation Set:**

20% of the training data will be used to form the validation set and the remaining 80% will be the new training set

In [2]:
import sklearn.model_selection

train_images, val_images, train_labels, val_labels = sklearn.model_selection.train_test_split(train_images_original, train_labels_original, 
                                                                                              test_size=0.20, random_state=42)
print('Training image shape:', train_images.shape)
print('Validation image shape:', val_images.shape)

Training image shape: (40000, 32, 32, 3)
Validation image shape: (10000, 32, 32, 3)


**Prepare the training and validation image data:**

In [0]:
train_images = train_images.astype('float32')/ 255
val_images = val_images.astype('float32')/ 255

**Prepare the labels:**

    airplane : 0     automobile : 1    bird : 2    cat : 3          deer : 4       dog : 5    frog : 6         horse : 7         ship : 8          truck : 9

In [4]:
import numpy as np
from keras.utils.np_utils import to_categorical   

train_labels = to_categorical(train_labels)
val_labels = to_categorical(val_labels)

print('Training label shape:', train_labels.shape)
print('Validation label shape:', val_labels.shape)

Training label shape: (40000, 10)
Validation label shape: (10000, 10)


**Convolutional Neural Network Architecture:**

In [15]:
from keras import models
from keras import layers

batch_size = 256
num_classes = 10
epochs = 50

network = models.Sequential()

network.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
network.add(layers.Conv2D(32, (3, 3), activation='relu'))
network.add(layers.MaxPooling2D(pool_size = (2, 2)))

network.add(layers.Conv2D(64, (3, 3), activation='relu'))
network.add(layers.Conv2D(64, (3, 3), activation='relu'))
network.add(layers.MaxPooling2D(pool_size = (2, 2)))

network.add(layers.Flatten())


network.add(layers.Dense(128, activation='relu'))
network.add(layers.Dropout(0.50))
network.add(layers.Dense(num_classes, activation='softmax'))
network.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 30, 30, 32)        896       
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 28, 28, 32)        9248      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 12, 12, 64)        18496     
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 10, 10, 64)        36928     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 1600)              0         
__________

**Initialize the Image Data Generator for Data Augmentation:**

In [0]:
from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator( rotation_range=15,
                 width_shift_range=0.1, height_shift_range=0.1,
                 horizontal_flip=True)
datagen.fit(train_images)

**Compile the network:**

In [0]:
network.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

**Train the network:**

In [17]:
network.fit_generator(datagen.flow(train_images, train_labels, batch_size=batch_size),steps_per_epoch=100, epochs=epochs, verbose=1)

Epoch 1/50
100/100 [==============================] - 14s 136ms/step - loss: 2.0902 - acc: 0.2349
Epoch 2/50
100/100 [==============================] - 13s 135ms/step - loss: 1.8585 - acc: 0.3272
Epoch 3/50
100/100 [==============================] - 14s 135ms/step - loss: 1.7237 - acc: 0.3786
Epoch 4/50
100/100 [==============================] - 13s 134ms/step - loss: 1.6471 - acc: 0.4033
Epoch 5/50
100/100 [==============================] - 13s 134ms/step - loss: 1.5498 - acc: 0.4419
Epoch 6/50
100/100 [==============================] - 14s 136ms/step - loss: 1.4999 - acc: 0.4620
Epoch 7/50
100/100 [==============================] - 13s 134ms/step - loss: 1.4432 - acc: 0.4886
Epoch 8/50
100/100 [==============================] - 14s 136ms/step - loss: 1.4107 - acc: 0.4989
Epoch 9/50
100/100 [==============================] - 13s 134ms/step - loss: 1.3523 - acc: 0.5207
Epoch 10/50
100/100 [==============================] - 13s 133ms/step - loss: 1.3266 - acc: 0.5326
Epoch 11/50
100/100

**Evaluate the network on the Validation Set:**

In [18]:
score = network.evaluate(val_images, val_labels, verbose=0)
print('Validation loss:', score[0])
print('Validation accuracy:', score[1]*100,"%")

Validation loss: 0.7468105646133423
Validation accuracy: 75.12 %
